In [161]:
import pandas as pd
from collections import deque
import ast

In [138]:
block = pd.read_csv(r"D:\gitworkflow\DT#25062020\pdfExtraction\block_aegon.csv")
line = pd.read_csv(r"D:\gitworkflow\DT#25062020\pdfExtraction\line_aegon.csv")
word = pd.read_csv(r"D:\gitworkflow\DT#25062020\pdfExtraction\word_aegon.csv")

In [81]:
def split_determiner(df):
    initial = True
    ind_list= deque()
    for ind,row in df.iterrows():
        if initial:
            initial=False
            prev_text = row['line']
            prev_size = row['size']
            prev_font_name = row['font_name']
            ind_list.append(ind)
        else:
            curr_text = row['line']
            curr_size = row['size']
            curr_font_name = row['font_name']
#             if all([len(prev_size)==1,len(prev_font_name)==1]):
            if all([prev_size!=curr_size,prev_font_name!=curr_font_name]):
                return True,ind_list
#             else:
#                 print('pass')
            ind_list.append(ind)
            prev_text = curr_text
            prev_size = curr_size
            prev_font_name = curr_font_name
    return False,None

In [87]:
def block_spliter(block : pd.DataFrame,
                line : pd.DataFrame,
                word : pd.DataFrame,width,height):
                
    block = block.sort_values(by=['bbox_top', 'bbox_left'])
    block.reset_index(inplace=True,drop=True)
    # aim is to see whether if a complete line cotains one kinda font
    line = line.sort_values(by=['bbox_top', 'bbox_left'])
    line.reset_index(inplace=True,drop=True)

    line_group = line.groupby(by=['block_no'])
    block_counter = -1
    
    for group_no , group_df in line_group:
        decider,index_list = split_determiner(group_df)
        if decider:
            # design block bbox coordinates
            #split bbox cordinates
            bbox_left = min(line.loc[index_list,'bbox_left'])
            bbox_top = min(line.loc[index_list,'bbox_top'])
            bbox_right = max(line.loc[index_list,'bbox_right'])
            bbox_bottom = max(line.loc[index_list,'bbox_bottom'])
            ########### WORK AROUND HERE FOR CHANGES
#             'bbox_left':left_bbox_block
#                 'bbox_top':top_bbox_block
#                 'bbox_right': block.loc[block_index,'bbox_right'],
#                 'bbox_bottom': block.loc[block_index,'bbox_bottom']
            #other split bbox coordinates right and bottom is same so working on left and top
            block_index = block[(block['block_no']==group_no)].index
            left_bbox_block = bbox_left
            top_bbox_block = bbox_bottom + (1*height)
            text = block.loc[block_index,'text'].str.split('|||')
            data = {
                'page_no': line['page_no'],
                'block_no': block_counter,
                'text': ' '.join(text[:len(index_list)]),
                'size' : block.loc[block_index,'size'][:len(index_list)],
                'font_name' : block.loc[block_index,'font_name'][:len(index_list)],
                'bbox_left':bbox_left,
                'bbox_top':bbox_top,
                'bbox_right': bbox_right,
                'bbox_bottom': bbox_bottom
            }
            
            #update other block info in block df
            block.loc[block_index,'size'] = block.loc[block_index,'size'][len(index_list):]
            block.loc[block_index,'font_name'] = block.loc[block_index,'font_name'][len(index_list):]            
            block.loc[block_index,'text'] = ' '.join(text[len(index_list):])
            block.loc[block_index, 'bbox_left'] = left_bbox_block
            block.loc[block_index,'bbox_top'] = top_bbox_block
            
            #append new block to block df
            block = block.append(data,ignore_index=True)
            ###########
            #rename the block_no in line dataframe
            line.loc[index_list,'block_no'] = block_counter
            
            #get the line numbers from dataframe
            line_no_ = deque(line.loc[index_list,'line_no'])
            # update the block number in words
            for _line_ in line_no_:
                word_index = deque(word[(word['block_no']==group_no) & (word['line_no']==_line_)].index)
                word.loc[word_index,'block_no'] = block_counter
            # increment the block counter    
            block_counter -= 1
            
            
    return block, line, word

In [88]:
block_spliter(block,line,word)

3 (False, None)
4 (False, None)
7 (True, deque([1]))
11 (False, None)
13 (True, deque([5]))
16 (False, None)
18 (True, deque([9]))
21 (False, None)
23 (True, deque([14, 15]))
24 (True, deque([21]))
25 (False, None)
26 (True, deque([28, 29]))
27 (True, deque([34]))


In [80]:
line

,page_no,block_no,line_no,line,size,font_name,bbox_left,bbox_top,bbox_right,bbox_bottom
0,0,4,0,BASE PROSPECTUS DATED 10 JULY 2018,[11.039999961853027],"['Times New Roman,Bold']",56.664001,78.494202,274.990021,93.784599
1,0,7,0,AEGON N.V.,[11.039999961853027],"['Times New Roman,Bold']",265.369995,165.974243,332.589996,181.264633
2,0,7,1,(incorporated with limited liability in The Ne...,"[11.039999961853027, 11.039999961853027]","['Times New Roman', 'Times New Roman,Italic']",177.529999,178.594849,420.460022,193.322205
3,0,7,2,and having its corporate seat in The Hague),"[11.039999961853027, 11.039999961853027]","['Times New Roman,Italic', 'Times New Roman']",199.970001,191.554871,398.020020,206.282227
4,0,11,0,and,[11.039999961853027],['Times New Roman'],289.750000,217.594849,308.350006,232.322205
5,0,13,0,AEGON FUNDING COMPANY LLC,[11.039999961853027],"['Times New Roman,Bold']",207.649994,244.004211,390.460022,259.294617
6,0,13,1,(organised under the laws of the State of Dela...,"[11.039999961853027, 11.039999961853027]","['Times New Roman', 'Times New Roman,Italic']",170.929993,256.624878,429.820007,271.352234
7,0,13,2,"and having its corporate seat in Wilmington, D...","[11.039999961853027, 11.039999961853027]","['Times New Roman,Italic', 'Times New Roman']",174.169998,269.584839,423.820007,284.312195
8,0,16,0,guaranteed by,[11.039999961853027],['Times New Roman'],266.570007,295.624878,331.390015,310.352234
9,0,18,0,AEGON N.V.,[11.039999961853027],"['Times New Roman,Bold']",265.369995,322.004211,332.589996,337.294617


In [79]:
line = line.sort_values(by=['bbox_top', 'bbox_left'])
line.reset_index(inplace=True,drop=True)

In [98]:
block

,page_no,block_no,text,size,font_name,bbox_left,bbox_top,bbox_right,bbox_bottom
0,0,4,BASE PROSPECTUS DATED 10 JULY 2018,[11.039999961853027],"['Times New Roman,Bold']",56.664001,78.494202,274.990021,93.784599
1,0,7,AEGON N.V. (incorporated with limited liabilit...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman,Bold', 'Times New Roman', 'T...",177.529999,165.974243,420.460022,206.282227
2,0,11,and,[11.039999961853027],['Times New Roman'],289.750000,217.594849,308.350006,232.322205
3,0,13,AEGON FUNDING COMPANY LLC (organised under the...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman,Bold', 'Times New Roman', 'T...",170.929993,244.004211,429.820007,284.312195
4,0,16,guaranteed by,[11.039999961853027],['Times New Roman'],266.570007,295.624878,331.390015,310.352234
5,0,18,AEGON N.V. (incorporated with limited liabilit...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman,Bold', 'Times New Roman', 'T...",177.529999,322.004211,420.460022,362.974640
6,0,21,Programme for the Issuance of Debt Instruments,"[11.039999961853027, 11.039999961853027]","['Times New Roman,Bold', 'Times New Roman,Bold']",228.649994,373.004211,369.309998,401.274628
7,0,23,Under this Programme for the Issuance of Debt ...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman', 'Times New Roman,Bold', 'T...",56.664001,410.604858,541.440002,503.332214
8,0,24,Notes issued by AFC will have the benefit of a...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman', 'Times New Roman,Bold', 'T...",56.664001,513.684875,541.245361,542.034668
9,0,25,The maximum aggregate nominal amount of all in...,"[11.039999961853027, 11.039999961853027, 11.03...","['Times New Roman', 'Times New Roman', 'Times ...",56.664001,551.634888,541.363220,618.442200


In [36]:
any([x==1,x==1])

True

In [84]:
x=deque([1,2])

In [89]:
min(line.loc[x,'bbox_left'])

177.52999877929688

In [90]:
x=line.copy()

In [92]:
x.loc[deque([1,2]),'bbox_left']=0

In [97]:
word[(word['block_no']==4) & (word['line_no']==0)].index()

TypeError: 'Int64Index' object is not callable

In [100]:
' '.join(deque(['1','2']))

'1 2'

In [103]:
block[block['block_no']==4].index

Int64Index([0], dtype='int64')

In [104]:
x=[{'x':1,'y':2},{'x':3,'y':4}]

In [106]:
y=pd.DataFrame(x)

In [108]:
y = y['y']*20

In [109]:
y

0    40
1    80
Name: y, dtype: int64

In [129]:
line_2 = block['text']

In [130]:
x=line_2.str.replace('|||',' ',regex=False)

In [135]:
x.loc[12]

'Under this Programme for the Issuance of Debt Instruments (theProgramme), each of Aegon N.V. (Aegon and, together with its consolidated subsidiaries,Aegon Group) and Aegon Funding Company LLC (AFC'

In [132]:
x

0                    BASE PROSPECTUS DATED 10 JULY 2018
1                                                   and
2                                         guaranteed by
3        Programme for the Issuance of Debt Instruments
4     The maximum aggregate nominal amount of all in...
5                                                     1
6                                            AEGON N.V.
7                                            AEGON N.V.
8                             AEGON FUNDING COMPANY LLC
9                             AEGON FUNDING COMPANY LLC
10                                           AEGON N.V.
11                                           AEGON N.V.
12    Under this Programme for the Issuance of Debt ...
13    Under this Programme for the Issuance of Debt ...
14    Notes issued by AFC will have the benefit of a...
15    Notes issued by AFC will have the benefit of a...
16    The Notes may be issued on a continuing basis ...
17    The Notes may be issued on a continuing ba

In [136]:
list(filter(lambda x: 'bold' in x.lower(),['hello','fontnamebold']))

['fontnamebold']

In [225]:
def split_determiner(df):
    initial = True
    ind_list= deque()
    for ind,row in df.iterrows():
        if initial:
            initial=False
            if type(row['font_name']) == list:
                prev_size = row['size']
                prev_font_name = row['font_name']
            else:
                prev_size = ast.literal_eval(row['size'])
                prev_font_name = ast.literal_eval(row['font_name'])
            prev_line = row['line']
            ind_list.append(ind)
            first_line = True
        else:
            if type(row['font_name']) == list:
                curr_size = row['size']
                curr_font_name = row['font_name']
            else:
                curr_size = ast.literal_eval(row['size'])
                curr_font_name = ast.literal_eval(row['font_name'])
            curr_line = row['line']
            if prev_font_name==curr_font_name:
                ind_list.append(ind)
            elif all([prev_size!=curr_size,prev_font_name!=curr_font_name]):
                if (len(prev_font_name)==1) and (len(list(filter(lambda x: 'bold' in x.lower(),prev_font_name)))>=1):
                    return True,ind_list
                elif len(list(filter(lambda x: 'bold' in x.lower(),prev_font_name)))>=1 and 'bold' in prev_font_name[0].lower() \
                                and ':' in prev_line.lower():
                    return True,ind_list
                elif  len(prev_size)>=1 and ':' in prev_line.lower():
                    if len(set(prev_size))>=2:
                        prev_set_size = sorted(set(prev_size),reverse=True)
                        if prev_size[0]==prev_set_size[0]:
                            return True,ind_list
                else:
                    pass
            prev_line = curr_line
            prev_size = curr_size
            prev_font_name = curr_font_name
    return False,None

In [226]:
def block_spliter(block : pd.DataFrame,
                line : pd.DataFrame,
                word : pd.DataFrame):
                
    block = block.sort_values(by=['bbox_top', 'bbox_left'])
    block.reset_index(inplace=True,drop=True)
    # aim is to see whether if a complete line cotains one kinda font
    line = line.sort_values(by=['bbox_top', 'bbox_left'])
    line.reset_index(inplace=True,drop=True)

    line_group = line.groupby(by=['block_no'])
    block_counter = -1
    
    for group_no , group_df in line_group:
        print(split_determiner(group_df))

In [227]:
block_spliter(block,line,word)

(False, None)
(False, None)
(True, deque([1]))
(False, None)
(True, deque([5]))
(False, None)
(True, deque([9]))
(False, None)
(False, None)
(False, None)
(False, None)
(False, None)
(False, None)


In [172]:
line = line.sort_values(by=['bbox_top', 'bbox_left'])
line.reset_index(inplace=True,drop=True)
line

,page_no,block_no,line_no,line,size,font_name,bbox_left,bbox_top,bbox_right,bbox_bottom
0,0,4,0,BASE PROSPECTUS DATED 10 JULY 2018,[11.039999961853027],"['Times New Roman,Bold']",56.664001,78.494202,274.990021,93.784599
1,0,7,0,AEGON N.V.,[11.039999961853027],"['Times New Roman,Bold']",265.369995,165.974243,332.589996,181.264633
2,0,7,1,(incorporated with limited liability in The Ne...,"[11.039999961853027, 11.039999961853027]","['Times New Roman', 'Times New Roman,Italic']",177.529999,178.594849,420.460022,193.322205
3,0,7,2,and having its corporate seat in The Hague),"[11.039999961853027, 11.039999961853027]","['Times New Roman,Italic', 'Times New Roman']",199.970001,191.554871,398.020020,206.282227
4,0,11,0,and,[11.039999961853027],['Times New Roman'],289.750000,217.594849,308.350006,232.322205
5,0,13,0,AEGON FUNDING COMPANY LLC,[11.039999961853027],"['Times New Roman,Bold']",207.649994,244.004211,390.460022,259.294617
6,0,13,1,(organised under the laws of the State of Dela...,"[11.039999961853027, 11.039999961853027]","['Times New Roman', 'Times New Roman,Italic']",170.929993,256.624878,429.820007,271.352234
7,0,13,2,"and having its corporate seat in Wilmington, D...","[11.039999961853027, 11.039999961853027]","['Times New Roman,Italic', 'Times New Roman']",174.169998,269.584839,423.820007,284.312195
8,0,16,0,guaranteed by,[11.039999961853027],['Times New Roman'],266.570007,295.624878,331.390015,310.352234
9,0,18,0,AEGON N.V.,[11.039999961853027],"['Times New Roman,Bold']",265.369995,322.004211,332.589996,337.294617


In [167]:
x.lower()

'hello'

In [182]:
x='hello'

In [187]:
type([x])==list

True

In [199]:
prev_font_name = ['bold','normal']
prev_line = 'emerging: hello world'

In [200]:
if len(list(filter(lambda x: 'bold' in x.lower(),prev_font_name)))>=1 and 'bold' in prev_font_name[0].lower() \
                                and ':' in prev_line.lower():
    print('hurrey')

hurrey


In [202]:
x=set([1,2])

In [204]:
len(x)

2

In [208]:
y=sorted(x)

In [210]:
y[0]

1

In [222]:
data =[
            {'line' : 'Emerging: Technologies are more','size' : [11.039999961853027,10.0],
            'font_name' : ['Times New Roman,','Times New Roman']},
            {'line' : 'organised under the laws of the State of Delaware, USA,','size' : [11.039999961853027, 11.039999961853027],
            'font_name' : ['Times New Roman', 'Times New Roman,Italic']}
        ]
df = pd.DataFrame(data)

In [223]:
df

,line,size,font_name
0,Emerging: Technologies are more,"[11.039999961853027, 10.0]","[Times New Roman,, Times New Roman]"
1,organised under the laws of the State of Delaw...,"[11.039999961853027, 11.039999961853027]","[Times New Roman, Times New Roman,Italic]"


In [224]:
split_determiner(df)

(True, deque([0]))

In [228]:
ast.literal_eval(['hhelo'])

ValueError: malformed node or string: ['hhelo']